In [1]:
import numpy as np

# 🧠 Solving the Infinite-Horizon HJB for Market Making

We compute the optimal bid and ask spreads that:

- Maximize expected utility of terminal wealth  
- Balance inventory risk and market opportunity  
- React to signals (price, imbalance, volatility)  

We solve a **stationary infinite-horizon HJB equation** to get the value function.


In [2]:
# No code needed for this high-level description

## 🧮 Step 1: Define Variables

### 🔢 State Variables (dynamically evolving)

Let the state be a tuple: $(t,q, s, z, \sigma)$

- $t$: Time  
- $q$: Inventory  
- $s$: Midprice  
- $z$: Order imbalance  
- $\sigma$: Volatility  

---

### ⚙️ Control Variables

- $\delta_b$: Bid spread from midprice → quote at $s - \delta_b$  
- $\delta_a$: Ask spread from midprice → quote at $s + \delta_a$


In [3]:
# Define state and control variables
q = 0  # inventory
s = 30000.0  # midprice
z = 0.0  # imbalance
sigma = 0.05  # volatility
delta_b = 0.5  # bid spread
delta_a = 0.5  # ask spread

### 📈 Market Parameters

- $\lambda_b(\delta_b) = A e^{-k \delta_b}$: Buy arrival rate  
- $\lambda_a(\delta_a) = A e^{-k \delta_a}$: Sell arrival rate  
- $\mu(t,s, z, \sigma)$: Drift of midprice  
- $\sigma$: Local volatility (state variable)


In [4]:
# Define market parameters
import numpy as np
A = 0.1
k = 1.5
gamma = 0.1
lambda_b = lambda delta: A * np.exp(-k * delta)
lambda_a = lambda delta: A * np.exp(-k * delta)

### 🎯 Utility and Value Function

We use exponential utility of terminal wealth:

$$
u(x, q, s) = -\exp\left(-\gamma\left(x + q s + \phi(t,q, s, z, \sigma)\right)\right)
$$

- $x$: Cash (drops out of control problem)  
- $\phi(t,q, s, z, \sigma)$: Value function to solve for


In [5]:
# Define the utility function and the ansatz
import math
def utility(x, q, s, phi):
    return -math.exp(-gamma * (x + q * s + phi))

## 📘 Step 2: The HJB Equation

In the stationary infinite-horizon case (no $\partial_t \phi$):

$$
0 = \mu(t,s, z, \sigma) \frac{\partial \phi}{\partial s}
+ \frac{1}{2} \sigma^2 \frac{\partial^2 \phi}{\partial s^2} + \\
\sup_{\delta_a} \lambda_a(\delta_a) \left[ u(t, x + (s + \delta_a), q - 1, s,\sigma) - u(t, x, q, s,\sigma) \right] + \\
\sup_{\delta_b} \lambda_b(\delta_b) \left[ u(t, x - (s - \delta_b), q + 1, s,\sigma) - u(t, x, q, s,\sigma) \right]
$$



## 📈 Including Volatility as a State Variable

Volatility $\sigma$ affects the risk of quoting and the optimal spread.

### ✅ Add volatility dynamics to the HJB:
If $\sigma$ follows an SDE like:
$$ d\sigma = \eta(\sigma)\,dt + \nu(\sigma)\,dW_t $$
Then the HJB gains two new terms:
$$ \eta(\sigma)\, \frac{\partial \phi}{\partial \sigma} + \frac{1}{2} \nu^2(\sigma) \frac{\partial^2 \phi}{\partial \sigma^2} $$

### ✅ The full HJB:
$$ \begin{aligned}
0 = &\ \mu(t,s, z, \sigma) \frac{\partial \phi}{\partial s}
+ \frac{1}{2} \sigma^2 \frac{\partial^2 \phi}{\partial s^2} \\
&+ \eta(\sigma) \frac{\partial \phi}{\partial \sigma}
+ \frac{1}{2} \nu^2(\sigma) \frac{\partial^2 \phi}{\partial \sigma^2} \\
&+ \sup_{\delta_a} \lambda_a(\delta_a) \left[ u(t, x + (s + \delta_a), q - 1, s,\sigma) - u(t, x, q, s,\sigma) \right] \\
&+\sup_{\delta_b} \lambda_b(\delta_b) \left[ u(t, x - (s - \delta_b), q + 1, s,\sigma) - u(t, x, q, s,\sigma) \right]
\end{aligned} $$

### ✅ Update optimal spreads:
$$ \delta_b^* = \phi(q+1, s, z, \sigma) - \phi(q, s, z, \sigma) + C $$
$$ \delta_a^* = -\left(\phi(q-1, s, z, \sigma) - \phi(q, s, z, \sigma)\right) + C $$

### ✅ Discretize $\sigma$:
- Use finite differences for $\partial_\sigma \phi$ and $\partial^2_{\sigma\sigma} \phi$
- Result: a 4D grid $\phi[q][s][z][\sigma]$


### ✏️ Plug in Exponential Utility

We assume exponential utility:
$$
u(t, x, q, s, \sigma) = -\exp\left(-\gamma \left( x + q s + \phi(q, s, z, \sigma) \right) \right)
$$

Let’s focus on the **ask jump term** in the HJB:
$$
\lambda_a(\delta_a) \left[ u(t, x + s + \delta_a, q - 1, s, \sigma) - u(t, x, q, s, \sigma) \right]
$$

Substitute the utility form:
$$
= \lambda_a(\delta_a) \left[
-\exp\left(-\gamma \left( x + s + \delta_a + (q - 1)s + \phi(q - 1, s, z, \sigma) \right) \right)
+ \exp\left(-\gamma \left( x + q s + \phi(q, s, z, \sigma) \right) \right)
\right]
$$

Factor out the common exponential:
$$
= -\lambda_a(\delta_a) \cdot \exp\left(-\gamma \left( x + q s + \phi(q, s, z, \sigma) \right) \right)
\left[
\exp\left( -\gamma \left( \delta_a - \Delta^- \phi \right) \right) - 1
\right]
$$

where we define:
$$
\Delta^- \phi := \phi(q, s, z, \sigma) - \phi(q - 1, s, z, \sigma)
$$

Let:
$$
F(\delta_a) := \lambda_a(\delta_a) \left[
\exp\left( -\gamma(\delta_a - \Delta^- \phi) \right) - 1
\right]
$$

Assume exponential fill intensity:
$$
\lambda_a(\delta_a) = A e^{-k \delta_a}
$$

Then:
$$
F(\delta_a) = A e^{-k \delta_a} \left[
e^{ -\gamma(\delta_a - \Delta^- \phi) } - 1
\right]
$$

Differentiate with respect to \( \delta_a \):
$$
\begin{aligned}
F'(\delta_a) &= \frac{d}{d\delta_a} \left[ A e^{-k \delta_a} \left( e^{ -\gamma(\delta_a - \Delta^- \phi) } - 1 \right) \right] \\
&= A \left[
- k e^{-k \delta_a} \left( e^{ -\gamma(\delta_a - \Delta^- \phi) } - 1 \right)
- \gamma e^{-k \delta_a} e^{ -\gamma(\delta_a - \Delta^- \phi) }
\right] \\
&= A e^{-k \delta_a} \left[
- (k + \gamma) e^{ -\gamma(\delta_a - \Delta^- \phi) } + k
\right]
\end{aligned}
$$

Set \( F'(\delta_a) = 0 \):
$$
(k + \gamma) e^{ -\gamma(\delta_a - \Delta^- \phi) } = k
$$

Take logs:
$$
- \gamma(\delta_a - \Delta^- \phi) = \log\left( \frac{k}{k + \gamma} \right)
\quad \Rightarrow \quad
\delta_a^* = \frac{1}{\gamma} \log\left( 1 + \frac{\gamma}{k} \right) - \Delta^- \phi
$$

Similarly, for the **bid jump term** we define:
$$
\Delta^+ \phi := \phi(q + 1, s, z, \sigma) - \phi(q, s, z, \sigma)
$$

and obtain:
$$
\delta_b^* = C + \Delta^+ \phi \\
Where,\ C = \frac{1}{\gamma} \log\left( 1 + \frac{\gamma}{k} \right)
$$

---




### 🧮 Finite Difference Approximations for Solving the HJB

We aim to solve the value function:
$$\phi(q, s, z, \sigma)$$
defined over a 4-dimensional grid.

Let:
- $q_i$: discrete inventory levels
- $s_j$: discrete midprice levels, with spacing $\Delta s$
- $z_k$: discrete order imbalance bins
- $\sigma_l$: discrete volatility levels, with spacing $\Delta \sigma$

Each point on the grid is denoted:
$$\phi_{i,j,k,l} := \phi(q_i, s_j, z_k, \sigma_l)$$

---

### 📘 Derivatives with Respect to Midprice $s$

**First derivative:**
$$
\left.\frac{\partial \phi}{\partial s}\right|_{i,j,k,l}
\approx \frac{\phi_{i,j+1,k,l} - \phi_{i,j-1,k,l}}{2\Delta s}
$$

**Second derivative:**
$$
\left.\frac{\partial^2 \phi}{\partial s^2}\right|_{i,j,k,l}
\approx \frac{\phi_{i,j+1,k,l} - 2\phi_{i,j,k,l} + \phi_{i,j-1,k,l}}{(\Delta s)^2}
$$

---

### 📘 Derivatives with Respect to Volatility $\sigma$

**First derivative:**
$$
\left.\frac{\partial \phi}{\partial \sigma}\right|_{i,j,k,l}
\approx \frac{\phi_{i,j,k,l+1} - \phi_{i,j,k,l-1}}{2\Delta \sigma}
$$

**Second derivative:**
$$
\left.\frac{\partial^2 \phi}{\partial \sigma^2}\right|_{i,j,k,l}
\approx \frac{\phi_{i,j,k,l+1} - 2\phi_{i,j,k,l} + \phi_{i,j,k,l-1}}{(\Delta \sigma)^2}
$$

---

### 📘 Differences in Inventory $q$

Since inventory evolves via discrete jumps, we use forward and backward differences rather than continuous derivatives.

**Forward difference (used in optimal bid calculation):**
$$\Delta^+ \phi(q_i) := \phi_{i+1,j,k,l} - \phi_{i,j,k,l}$$

**Backward difference (used in optimal ask calculation):**
$$\Delta^- \phi(q_i) := \phi_{i,j,k,l} - \phi_{i-1,j,k,l}$$

In [14]:
# Example jump terms computation
phi_q = 1.0
phi_q_plus1 = 1.05
phi_q_minus1 = 0.95
delta_b = phi_q_plus1 - phi_q + (1/gamma) * np.log((k+gamma)/k)
delta_a = -(phi_q_minus1 - phi_q) + (1/gamma) * np.log((k+gamma)/k)
J_bid = lambda_b(delta_b) * (np.exp(-gamma * (delta_b + phi_q_plus1 - phi_q)) - 1)
J_ask = lambda_a(delta_a) * (np.exp(-gamma * (-delta_a + phi_q_minus1 - phi_q)) - 1)

In [15]:
# Compute optimal spreads
C = (1/gamma) * np.log((k + gamma)/k)
delta_b_star = phi_q_plus1 - phi_q + C
delta_a_star = -(phi_q_minus1 - phi_q) + C

### 🧮 Optimal Spreads

$$
\delta_b^* = \phi(q+1, s, z, \sigma) - \phi(q, s, z, \sigma) + C
$$

$$
\delta_a^* = -\left(\phi(q-1, s, z, \sigma) - \phi(q, s, z, \sigma)\right) + C
$$

Where:

$$
C = \frac{1}{\gamma} \log\left(\frac{k + \gamma}{k}\right)
$$


In [16]:
# Basic value iteration loop
phi_grid = np.zeros((3, 3, 3, 3))  # Example shape (q, s, z, sigma)
for iter in range(10):
    # dummy update rule
    phi_grid += 0.01  # Increment as a placeholder

## 🔧 Step 3: Numerical Value Iteration

1. Discretize $t,q, s, z, \sigma$  
2. Initialize grid $\phi[t][q][s][z][\sigma]$  
3. Repeat until convergence:
   - For each grid point:
     - Compute $\delta_b, \delta_a$
     - Compute jump terms $J_{\text{bid}}, J_{\text{ask}}$
     - Approximate $\partial_s \phi$, $\partial^2_{ss} \phi$, $\partial_\sigma \phi$ and $\partial^2_{\sigma\sigma}$ via finite differences
     - Evaluate HJB:
     
       $$
       HJB = \mu \frac{\partial \phi}{\partial s} + \frac{1}{2} \sigma^2 \frac{\partial^2 \phi}{\partial s^2} + \eta(\sigma)\, \frac{\partial \phi}{\partial \sigma} + \frac{1}{2} \nu^2(\sigma) \frac{\partial^2 \phi}{\partial \sigma^2} + J_{\text{bid}} + J_{\text{ask}}
       $$

     - Update:

       $$
       \phi_{\text{new}} = \phi - \alpha \cdot HJB
       $$
4. Measure:

   $$
   \text{max\_diff} = \max \left|\phi_{\text{new}} - \phi\right|
   $$

   Stop if $\text{max\_diff} < \varepsilon$





In [10]:
# Extract control policy example
def optimal_spreads(phi, q, s, z, sigma):
    C = (1/gamma) * np.log((k + gamma)/k)
    delta_b = phi[q+1, s, z, sigma] - phi[q, s, z, sigma] + C
    delta_a = -(phi[q-1, s, z, sigma] - phi[q, s, z, sigma]) + C
    return delta_b, delta_a

## 📤 Step 4: Extract Control Policy

After convergence:

$$
\delta_b(q, s, z, \sigma) = \phi(q+1) - \phi(q) + C
$$

$$
\delta_a(q, s, z, \sigma) = -(\phi(q-1) - \phi(q)) + C
$$


### 🎯 Final Expressions for Optimal Quotes

\[
\boxed{
\delta_b^* = \frac{1}{\gamma} \log\left( 1 + \frac{\gamma}{k} \right) + \left( \phi(q + 1, s, z, \sigma) - \phi(q, s, z, \sigma) \right)
}
\]

\[
\boxed{
\delta_a^* = \frac{1}{\gamma} \log\left( 1 + \frac{\gamma}{k} \right) - \left( \phi(q, s, z, \sigma) - \phi(q - 1, s, z, \sigma) \right)
}
\]

In [21]:
import numpy as np

# Define the state variables (excluding time 't' for stationary HJB)
state_variables = ['s', 'vol', 'z', 'q']

# Create a 4D value grid (100 points along each axis)
discretised_grids = np.zeros((100, 100, 100, 100))

# Define uniform grid range from 0 to 100 (inclusive)
grid_range = np.array(range(0, 101, 1))  # 101 points from 0 to 100

# Create a dictionary to hold the range for each state variable
variable_grids = {}

# Assign the same range to each variable for now
for var in state_variables:
    variable_grids[var] = grid_range

# Display the constructed grid ranges
for var in state_variables:
    print(f"{var} grid: {variable_grids[var]}")


s grid: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100]
vol grid: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100]
z grid: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  